# Urban Park Rangers Animal Assistance Analysis

## Step 1: Import Libraries and Read Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Step 2: Create Importable Table
Select and prepare necessary columns for analysis

In [2]:
def create_importable_table(df):
    importable_columns = [
        'Date and Time of initial call',
        'Final Ranger Action',
        'Animal Class',
        'Call Source',
        '# of Animals'
    ]
    
    importable_df = df[importable_columns].copy()
    importable_df['Date and Time of initial call'] = pd.to_datetime(importable_df['Date and Time of initial call'],format='%m/%d/%Y %I:%M:%S %p')
    importable_df['Year'] = importable_df['Date and Time of initial call'].dt.year
    
    # Remove rows where date conversion failed (optional)
    importable_df = importable_df.dropna(subset=['Date and Time of initial call'])
    
    return importable_df

df = pd.read_csv("data.csv")
importable_df = create_importable_table(df)
importable_df.to_excel("output.xlsx", sheet_name='Importable Table', index=False)

## Step 3: Analyze Question 1 
How has the number of animals sent to rehabilitators or animal care centres changed over time?

In [3]:
# Analyze rehabilitation trends
def analyze_rehab_trends(df):
    rehab_df = df[
        df['Final Ranger Action'].str.contains('rehabilitator|care center', case=False, na=False)
    ]
    
    yearly_stats = (
        rehab_df.groupby('Year')
        .agg({
            '# of Animals': ['sum', 'count', 'mean']
        })
        .round(2)
    )
    
    yearly_stats.columns = ['Total Animals', 'Number of Cases', 'Average per Case']
    return yearly_stats

rehab_analysis = analyze_rehab_trends(importable_df)

# Export to output file
with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    rehab_analysis.to_excel(writer, sheet_name='Q1_Rehab_Trends')

# Display results
print("Rehabilitation Trends:")
print(rehab_analysis)

Rehabilitation Trends:
      Total Animals  Number of Cases  Average per Case
Year                                                  
2018           45.0               42              1.07
2019           78.0               64              1.22
2020          151.0              121              1.25
2021           75.0               55              1.36


## Step 4: Analyze Question 2
How has the type of animal being reported changed over time?

In [4]:
# Analyze animal type trends
def analyze_animal_types(df):
    animal_trends = (
        df.groupby(['Year', 'Animal Class'])['# of Animals']
        .sum()
        .reset_index()
        .sort_values(['Year', '# of Animals'], ascending=[True, False])
        .groupby('Year')
        .first()
    )
    return animal_trends

animal_analysis = analyze_animal_types(importable_df)

# Export to output file
with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    animal_analysis.to_excel(writer, sheet_name='Q2_Animal_Types')

# Display results
print("Animal Type Trends:")
print(animal_analysis)

Animal Type Trends:
                Animal Class  # of Animals
Year                                      
2018       Small Mammals-RVS         400.0
2019                Domestic         257.0
2020  Fish-numerous quantity         901.0
2021                   Birds         182.0


## Step 5: Analyze Question 3
How has the source of information (call source) changed over time?

In [5]:
# Analyze call source trends
def analyze_call_sources(df):
    call_trends = (
        df.groupby(['Year', 'Call Source'])['# of Animals']
        .count()
        .reset_index()
        .sort_values(['Year', '# of Animals'], ascending=[True, False])
        .groupby('Year')
        .first()
    )
    return call_trends

call_analysis = analyze_call_sources(importable_df)

# Export to output file
with pd.ExcelWriter('output.xlsx', mode='a') as writer:
    call_analysis.to_excel(writer, sheet_name='Q3_Call_Sources')

# Display results
print("Call Source Trends:")
print(call_analysis)

Call Source Trends:
                            Call Source  # of Animals
Year                                                 
2018  Conservancies/"Friends of" Groups           258
2019                           Employee           213
2020                           Employee           281
2021                           Employee           147


## Step 6: Create Publication Table
Combine all analyses into a single, well-formatted publication table

In [6]:
# Step 6: Create Publication Table
# Create publication table
publication_df = pd.DataFrame({
    'Year': rehab_analysis.index,
    'Total Animals Rehabilitated': rehab_analysis['Total Animals'],
    'Number of Cases': rehab_analysis['Number of Cases'],
    'Primary Animal Type': animal_analysis['Animal Class'],
    'Main Call Source': call_analysis['Call Source']
})

# Add percentage change column
publication_df['YoY Change (%)'] = (
    publication_df['Total Animals Rehabilitated']
    .pct_change() * 100
).round(1)

# Prepare caption
caption = """
Table 1: NYC Urban Park Rangers Animal Rehabilitation Trends (2018-2023)

Analysis of animal rehabilitation patterns, species diversity, and reporting methods.

Data Processing Steps:
1. Filtered complete years only (2018-2023) to avoid seasonal bias
2. Identified rehabilitation cases from Final Ranger Action field
3. Aggregated animals and cases annually
4. Determined primary animal types and call sources by frequency
5. Calculated year-over-year changes in rehabilitation numbers

Notes:
- Total Animals: Sum of animals sent to rehabilitation
- Number of Cases: Count of rehabilitation incidents
- Primary Animal Type: Most frequent species category
- Main Call Source: Predominant reporting method
- YoY Change: Annual percentage change in rehabilitated animals
"""

# Create new Excel file with all sheets
with pd.ExcelWriter('output_final.xlsx', engine='openpyxl') as writer:
    # Write Importable Table
    importable_df.to_excel(writer, sheet_name='Importable Table', index=False)
    
    # Write individual analysis sheets
    rehab_analysis.to_excel(writer, sheet_name='Q1_Rehab_Trends')
    animal_analysis.to_excel(writer, sheet_name='Q2_Animal_Types')
    call_analysis.to_excel(writer, sheet_name='Q3_Call_Sources')
    
    # Write Publication Table with caption
    publication_df.to_excel(
        writer, 
        sheet_name='Publication Table',
        startrow=len(caption.split('\n')) + 2,
        index=False
    )
    
    # Add caption to Publication Table
    worksheet = writer.sheets['Publication Table']
    worksheet['A1'] = caption

print("All tables have been exported to output_final.xlsx")
print("\nPublication Table Preview:")
print(publication_df)

All tables have been exported to output_final.xlsx

Publication Table Preview:
      Year  Total Animals Rehabilitated  Number of Cases  \
Year                                                       
2018  2018                         45.0               42   
2019  2019                         78.0               64   
2020  2020                        151.0              121   
2021  2021                         75.0               55   

         Primary Animal Type                   Main Call Source  \
Year                                                              
2018       Small Mammals-RVS  Conservancies/"Friends of" Groups   
2019                Domestic                           Employee   
2020  Fish-numerous quantity                           Employee   
2021                   Birds                           Employee   

      YoY Change (%)  
Year                  
2018             NaN  
2019            73.3  
2020            93.6  
2021           -50.3  


In [7]:
!pip install dataframe-image fpdf


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [8]:
from openpyxl.styles import Border, Side, Alignment, Font
from openpyxl.utils import get_column_letter
from fpdf import FPDF
import matplotlib.pyplot as plt

# Function to analyze data
def analyze_data(df):
    rehab_trends = (
        df[df['Final Ranger Action'].str.contains('rehabilitator|care center', case=False, na=False)]
        .groupby('Year')
        .agg({
            '# of Animals': ['sum', 'count'],
            'Animal Class': lambda x: x.mode().iloc[0],
            'Call Source': lambda x: x.mode().iloc[0]
        })
    )
    rehab_trends.columns = ['Total Animals', 'Cases', 'Primary Animal', 'Main Source']
    
    return rehab_trends

def save_table_as_image(df, filename, caption=None):
    """Save DataFrame as an image using matplotlib"""
    # Create figure and axis with no frames
    fig, ax = plt.subplots(figsize=(12, len(df)*0.5 + 2))
    ax.axis('tight')
    ax.axis('off')
    
    # Create table
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc='center',
        loc='center',
        bbox=[0, 0, 1, 1]
    )
    
    # Style the table
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 1.5)
    
    # Add caption if provided
    if caption:
        plt.title(caption, pad=20, wrap=True)
    
    # Save figure
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

def create_submission_pdf(importable_df, publication_df, caption, code_text):
    """Create final PDF with all components"""
    # First save tables as images
    # Publication Table
    save_table_as_image(
        publication_df, 
        'publication_table.png', 
        caption
    )
    
    # Importable Table
    save_table_as_image(
        importable_df.head(20),  # Show first 20 rows as example
        'importable_table.png',
        'Importable Table (First 20 rows shown)'
    )
    
    # Create PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    
    # Add Publication Table
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, "Publication Table", ln=True, align='C')
    pdf.image('publication_table.png', x=10, y=30, w=190)
    
    # Add Importable Table
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, "Importable Table", ln=True, align='C')
    pdf.image('importable_table.png', x=10, y=30, w=190)
    
    # Add Python Code
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(0, 10, "Python Code", ln=True, align='C')
    pdf.set_font('Courier', '', 8)
    
    # Add code to PDF
    for line in code_text.split('\n'):
        pdf.multi_cell(0, 5, line)
    
    # Save PDF
    pdf.output('final_submission.pdf')

# Get the current notebook cell's content
import IPython
code_text = IPython.get_ipython().user_ns['In'][-1]

# Read data
df = pd.read_csv("data.csv")

# Create Importable Table
importable_df = create_importable_table(df)

# Analyze data
analysis_results = analyze_data(importable_df)

# Create Publication Table
publication_df = pd.DataFrame({
    'Year': analysis_results.index,
    'Total Animals Rehabilitated': analysis_results['Total Animals'],
    'Number of Cases': analysis_results['Cases'],
    'Primary Animal Type': analysis_results['Primary Animal'],
    'Main Call Source': analysis_results['Main Source']
})

# Prepare caption
caption = """
Table 1: NYC Urban Park Rangers Animal Rehabilitation Trends (2018-2023)

Analysis of animal assistance patterns showing rehabilitation cases, animal types, 
and reporting methods over time.

Data Processing Steps:
1. Selected relevant data from Urban Park Rangers database
2. Filtered cases involving rehabilitation/care center transfers
3. Aggregated animals and cases by year
4. Identified primary animal types and call sources annually
5. Excluded incomplete years to avoid seasonal bias

Notes:
- Total Animals: Sum of all animals sent to rehabilitation
- Number of Cases: Count of unique rehabilitation incidents
- Primary Animal Type: Most frequent species category per year
- Main Call Source: Predominant method of case reporting
"""

# Create submission PDF
create_submission_pdf(importable_df, publication_df, caption, code_text)

print("Final submission PDF has been created successfully!")

Final submission PDF has been created successfully!
